## Preparación inicial

In [1]:
import pandas as pd
import requests

In [2]:
xls = pd.ExcelFile('Prueba.xlsx')

In [3]:
sheet_names = xls.sheet_names
sheet_names

['Instrucciones', 'Glosario', 'PuntoPartida', 'PuntoLlegada', 'Siniestros']

In [4]:
punto_partida_df = pd.read_excel(xls, 'PuntoPartida')
punto_llegada_df = pd.read_excel(xls, 'PuntoLlegada')
siniestros_df = pd.read_excel(xls, 'Siniestros')

### Análisis Exploratorio
* Las coordenadas en ```punto_partida_df``` y ```punto_llegada_df``` están escritas en la notación *decimal degrees*.
* Las latitudes deben de ser números entre -90 y 90 grados.
* Las longitudes deben de ser números entre -180 y 180 grados.

#### punto_partida_df

In [5]:
punto_partida_df.dtypes

IDRuta                   object
NombreRuta               object
TipoUbicacion            object
Empresa                  object
HoraProgramadaSalida     object
Longitud                float64
Latitud                 float64
ValorMercancia            int64
IDConductor               int64
dtype: object

In [6]:
punto_partida_df.head()

,IDRuta,NombreRuta,TipoUbicacion,Empresa,HoraProgramadaSalida,Longitud,Latitud,ValorMercancia,IDConductor
0,a1,Relleno de Inventario,Bodega,Empresa 1,10:00:00,-100.316540,25.696300,150000,1
1,b2,Traslado de efectivo,Tienda,Empresa 1,09:00:00,-100.203823,25.663144,300000,1
2,c3,Relleno de Inventario,Bodega,Empresa 3,14:00:00,-100.145293,25.788839,1500000,1
3,d4,Relleno de Inventario,Bodega,Empresa 2,11:00:00,-100.367606,25.814914,200000,1
4,e5,Traslado de efectivo,Tienda,Empresa 3,15:00:00,-100.421277,25.744508,123000,1


#### punto_llegada_df

In [7]:
punto_llegada_df.dtypes

IDRuta             object
NombreRuta         object
TipoUbicacion      object
Empresa            object
HoraSalida         object
Longitud           object
Latitud           float64
ValorMercancia      int64
IDConductor         int64
dtype: object

* La columna ```longitud``` debería ser de tipo ```float64```.

In [8]:
punto_llegada_df.head()

,IDRuta,NombreRuta,TipoUbicacion,Empresa,HoraSalida,Longitud,Latitud,ValorMercancia,IDConductor
0,a1,Relleno de Inventario,Tienda,Empresa 1,10:03:00,-100.228542,25.550476,150000,1
1,b2,Traslado de efectivo,Banco,Banco 1,08:58:00,"-100.318156048963,",25.685163,300000,1
2,c3,Relleno de Inventario,Tienda,Empresa 3,14:10:00,-100.242414,25.664271,1500000,1
3,d4,Relleno de Inventario,Tienda,Empresa 2,11:00:19,-100.381548,25.664200,200000,1
4,e5,Traslado de efectivo,Banco,Banco 1,16:00:00,-100.318156,25.685163,123000,1


* Un valor en la columna *Longitud* no es un float valido pues termina con una coma.

In [9]:
# Remove any commas from the 'Longitud' column
punto_llegada_df['Longitud'] = punto_llegada_df['Longitud'].astype(str).str.replace(',', '', regex=False)

# Optionally, convert the cleaned column back to numeric if it's supposed to be numbers
punto_llegada_df['Longitud'] = pd.to_numeric(punto_llegada_df['Longitud'], errors='coerce')

In [10]:
punto_llegada_df.head()

,IDRuta,NombreRuta,TipoUbicacion,Empresa,HoraSalida,Longitud,Latitud,ValorMercancia,IDConductor
0,a1,Relleno de Inventario,Tienda,Empresa 1,10:03:00,-100.228542,25.550476,150000,1
1,b2,Traslado de efectivo,Banco,Banco 1,08:58:00,-100.318156,25.685163,300000,1
2,c3,Relleno de Inventario,Tienda,Empresa 3,14:10:00,-100.242414,25.664271,1500000,1
3,d4,Relleno de Inventario,Tienda,Empresa 2,11:00:19,-100.381548,25.664200,200000,1
4,e5,Traslado de efectivo,Banco,Banco 1,16:00:00,-100.318156,25.685163,123000,1


In [11]:
punto_llegada_df.dtypes

IDRuta             object
NombreRuta         object
TipoUbicacion      object
Empresa            object
HoraSalida         object
Longitud          float64
Latitud           float64
ValorMercancia      int64
IDConductor         int64
dtype: object

#### siniestros_df

In [12]:
siniestros_df.dtypes

FolioSiniestro       int64
Evento              object
Subtipo             object
UnidadOperativa     object
IDTienda           float64
IDVehiculo         float64
ValorPerdido         int64
Latitud            float64
Longitud           float64
dtype: object

In [13]:
siniestros_df.head()

,FolioSiniestro,Evento,Subtipo,UnidadOperativa,IDTienda,IDVehiculo,ValorPerdido,Latitud,Longitud
0,10001415,Robo,Con violencia,Tienda,4.0,NaN,84723,25.808908,-100.379278
1,10001213,Robo,Con violencia,Vehiculo,NaN,2.0,117041,25.799021,-100.386182
2,10001224,Robo,Sin violencia,Vehiculo,NaN,2.0,116400,25.797738,-100.381691
3,10001327,Robo,Sin violencia,Vehiculo,NaN,3.0,117888,25.786821,-100.379681
4,10001316,Robo,Con violencia,Tienda,3.0,NaN,73533,25.777179,-100.180762


## Calcular driving distance

In [14]:
rutas_df = pd.merge(
    punto_partida_df,
    punto_llegada_df,
    on=['IDRuta', 'NombreRuta', 'ValorMercancia', 'IDConductor'],
    suffixes=('Partida', 'Llegada')  # To differentiate columns if any aren't shared
)

# Show the merged DataFrame
rutas_df

,IDRuta,NombreRuta,TipoUbicacionPartida,EmpresaPartida,HoraProgramadaSalida,LongitudPartida,LatitudPartida,ValorMercancia,IDConductor,TipoUbicacionLlegada,EmpresaLlegada,HoraSalida,LongitudLlegada,LatitudLlegada
0,a1,Relleno de Inventario,Bodega,Empresa 1,10:00:00,-100.316540,25.696300,150000,1,Tienda,Empresa 1,10:03:00,-100.228542,25.550476
1,b2,Traslado de efectivo,Tienda,Empresa 1,09:00:00,-100.203823,25.663144,300000,1,Banco,Banco 1,08:58:00,-100.318156,25.685163
2,c3,Relleno de Inventario,Bodega,Empresa 3,14:00:00,-100.145293,25.788839,1500000,1,Tienda,Empresa 3,14:10:00,-100.242414,25.664271
3,d4,Relleno de Inventario,Bodega,Empresa 2,11:00:00,-100.367606,25.814914,200000,1,Tienda,Empresa 2,11:00:19,-100.381548,25.664200
4,e5,Traslado de efectivo,Tienda,Empresa 3,15:00:00,-100.421277,25.744508,123000,1,Banco,Banco 1,16:00:00,-100.318156,25.685163


## Calculate Driving Distance

In [15]:
def get_driving_distance(start_coords, end_coords):
    # Unpack the coordinates
    start_lon, start_lat = start_coords
    end_lon, end_lat = end_coords

    # OSRM API endpoint
    url = f"http://router.project-osrm.org/route/v1/driving/{start_lon},{start_lat};{end_lon},{end_lat}?overview=false"

    # Send the request to OSRM API
    response = requests.get(url)
    
    # Parse the JSON response
    data = response.json()

    if data and 'routes' in data:
        # Extract the distance in meters
        distance = data['routes'][0]['distance']
        # Extract the duration in seconds
        duration = data['routes'][0]['duration']
        return distance, duration
    else:
        return None, None

# Example usage with coordinates from your spreadsheet
start_coords = (-100.31654, 25.6963)  # Starting point coordinates
end_coords = (-100.228542, 25.550476)  # Destination point coordinates

distance, duration = get_driving_distance(start_coords, end_coords)
print(f"Driving Distance: {distance} meters, Duration: {duration} seconds")

Driving Distance: 20482.1 meters, Duration: 1054.3 seconds


In [16]:
# Apply function to each row in rutas_df
def apply_driving_distance(row):
    # Extract the starting and ending coordinates
    start_coords = (row['LongitudPartida'], row['LatitudPartida'])
    end_coords = (row['LongitudLlegada'], row['LatitudLlegada'])
    
    # Get the driving distance and duration
    distance, duration = get_driving_distance(start_coords, end_coords)
    
    return pd.Series([distance, duration], index=['DistanceMeters', 'DurationSeconds'])

# Apply the function to every row and store results in new columns
rutas_df[['DistanceMeters', 'DurationSeconds']] = rutas_df.apply(apply_driving_distance, axis=1)

In [17]:
rutas_df

,IDRuta,NombreRuta,TipoUbicacionPartida,EmpresaPartida,HoraProgramadaSalida,LongitudPartida,LatitudPartida,ValorMercancia,IDConductor,TipoUbicacionLlegada,EmpresaLlegada,HoraSalida,LongitudLlegada,LatitudLlegada,DistanceMeters,DurationSeconds
0,a1,Relleno de Inventario,Bodega,Empresa 1,10:00:00,-100.316540,25.696300,150000,1,Tienda,Empresa 1,10:03:00,-100.228542,25.550476,20482.1,1054.3
1,b2,Traslado de efectivo,Tienda,Empresa 1,09:00:00,-100.203823,25.663144,300000,1,Banco,Banco 1,08:58:00,-100.318156,25.685163,13728.1,1052.4
2,c3,Relleno de Inventario,Bodega,Empresa 3,14:00:00,-100.145293,25.788839,1500000,1,Tienda,Empresa 3,14:10:00,-100.242414,25.664271,23049.7,1570.3
3,d4,Relleno de Inventario,Bodega,Empresa 2,11:00:00,-100.367606,25.814914,200000,1,Tienda,Empresa 2,11:00:19,-100.381548,25.664200,25139.5,1597.2
4,e5,Traslado de efectivo,Tienda,Empresa 3,15:00:00,-100.421277,25.744508,123000,1,Banco,Banco 1,16:00:00,-100.318156,25.685163,14105.0,928.2


## Visualizar ```siniestros_df```

In [18]:
siniestros_df.tail(10)

,FolioSiniestro,Evento,Subtipo,UnidadOperativa,IDTienda,IDVehiculo,ValorPerdido,Latitud,Longitud
45,10001412,Robo,Sin violencia,Tienda,4.0,NaN,23706,25.673092,-100.230099
46,10001240,Vandalismo,NaN,Tienda,2.0,NaN,6217,25.672449,-100.317756
47,10001144,Vandalismo,NaN,Vehiculo,NaN,1.0,56972,25.669278,-100.235860
48,10001151,Delincuencia Organizada,NaN,Tienda,1.0,NaN,0,25.668250,-100.303949
49,10001119,Robo,Con violencia,Vehiculo,NaN,1.0,7348,25.668106,-100.317987
50,10001154,Delincuencia Organizada,NaN,Tienda,1.0,NaN,0,25.665895,-100.299721
51,10001133,Robo,Sin violencia,Vehiculo,NaN,1.0,63773,25.664839,-100.237429
52,10001152,Delincuencia Organizada,NaN,Tienda,1.0,NaN,0,25.653002,-100.291979
53,10001223,Robo,Con violencia,Tienda,2.0,NaN,8058,25.619191,-100.270167
54,1000143,Robo,Con violencia,Tienda,4.0,NaN,2022,25.597487,-100.260996
